In [65]:
# ==================================================================
# Script to analyse tractography-related statistics
# ==================================================================
import os
import sys
import glob #for wildcard matching
import collections #for ordered dictionary
import time
import subprocess
import nibabel as nib
import numpy as np

In [66]:
"""Anthony's notes
- modules that needs to be loaded:
    - qbatch/git
    - FSL

- during the initiation of the idx and pct file paths, it is ASSUMED that all input paths will:
    - have a perfect correspondence of pct to idx paths for each subject
        - both will appear in the same ordered after being sorted in the lists
    - the second-to-last part deliminated by "/" will be the subject ID for both files
- input file path assumptions
    - there will be no period character (".") OTHER than for at the end
    - there are two total period characters: "filename.nii.gz"
"""
print "All is well"

All is well


In [67]:
# ==================================================================
# Setup files and directories
# ==================================================================

""" Note on directory hierarchy for the input, reference and transform files 
parent_dir
    |
    +-- subj_dir_1 (as indicated by dir_wc)
    |        |
    |        +- file_of_interest (as indicated by *_file_name)
    |
    +-- subj_dir_2 ...
"""

#Variable indication how many component parcellation we want to use
SEG_NUM = 5 
# ========================================
# Input files for fslmaths and fslstats
# ========================================
in_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/seg'+str(SEG_NUM)+'_tract2voxel_probability_labels/model_space'
in_dir_wc='[0-9][0-9][0-9][0-9][0-9][0-9]'
idx_file_name = '*_region_seg_idx_modelSpace.nii.gz'
pct_file_name = '*_region_seg_pct_modelSpace.nii.gz' 

# ========================================
# Intermediate files
# ========================================
inter_parent_dir='/data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/seg'+str(SEG_NUM)+'_tract2voxel_probability_labels/tractAnalysis_model_space'
csvDump_filename="seg"+str(SEG_NUM)+"_avgComponentWinPct_dump.csv"

# ========================================
# JobDoc files
# ========================================
jobDoc_dir='/data/chamal/projects/anthony/qbatch_jobDocs/track_analysis_related'
jobList_path=os.path.join(jobDoc_dir,'autoSubmitted_seg%d_tractAnalysis.sub'%SEG_NUM)
jobScript_stdev_name=''

In [68]:
# ==================================================================
# Initiate paths of the two input files
# ==================================================================
"""Dictionary hierarchy
subjFile_Paths
    |
    ["100307"]
    |    |
    |    ["idx"] -- path to 100307's idx file
    |    ["pct"] -- path to 100307's pct file
    |
    ["100408"]
    ...
"""
#Find all files on paths
idx_file_list=glob.glob(os.path.join(in_parent_dir,in_dir_wc,idx_file_name))
pct_file_list=glob.glob(os.path.join(in_parent_dir,in_dir_wc,pct_file_name))
#Sort the two lists
idx_file_list.sort()
pct_file_list.sort()
#Store the two files into a sorted dictionary
subjFile_paths = collections.OrderedDict([]) #Initiate ordered dictionary
for idx_file_path, pct_file_path in zip(idx_file_list, pct_file_list):
    #Store the file paths in a temp dictionary
    temp_dict = {}
    temp_dict['idx'] = idx_file_path
    temp_dict['pct'] = pct_file_path
    #Check for subject ID and that they match up
    temp_subj_ID = idx_file_path.split('/')[-2]
    if pct_file_path.split('/')[-2] != temp_subj_ID: #If the two file IDs do not match
        print "Something went wrong in the pct and idx file matching for ID: %s" % (temp_subj_ID)
    #Add the temporary dictionary to the sorted dictionary
    subjFile_paths[temp_subj_ID] = temp_dict

#Let user know 
print "The parcellation component quantity is: %d" % SEG_NUM
print "idx_file_list length: %d" % len(idx_file_list)
print "pct_file_list length: %d" % len(pct_file_list)
print "\n%d subjects were initiated into the subjFile_Paths dictionary:" % len(subjFile_paths)
for key in subjFile_paths:
    print key,
print "\n\nHopefully the above looks right!"

The parcellation component quantity is: 5
idx_file_list length: 100
pct_file_list length: 100

100 subjects were initiated into the subjFile_Paths dictionary:
100307 100408 101006 101107 101410 101915 102008 102311 102816 103111 105014 105115 105216 106016 106521 107321 107422 108121 108323 108525 109123 110411 111312 111413 111716 113215 113821 113922 114419 115320 116524 117122 118528 118730 118932 119833 120212 120515 121618 122317 122620 123117 123420 123925 124422 125525 126325 126628 127630 127933 128127 128632 129028 130013 130316 131217 131722 132118 133019 133827 134324 135225 135528 136227 136833 137027 138534 139233 140925 144832 146432 147030 147737 148840 148941 149741 150423 150524 150726 151223 151526 151627 151728 153833 154734 154835 154936 156233 156637 157437 158136 159239 159340 160830 161327 161630 162228 162733 163331 164030 

Hopefully the above looks right!


In [64]:
# ==================================================================
# Generate the jobs needed to calculate everyone's component average
# ==================================================================

#Qbatch specifics
SUBMIT = False
CHUNKSIZE = '3' #Need each 3 lines to run sequentially 
WALLTIME = '00:30:00'


#Check to see if the output parent directory is present, if not create it
if os.path.isdir(inter_parent_dir):
    print "Intermediate directory found: %s" % inter_parent_dir
else:
    os.mkdir(inter_parent_dir)
    print "Intermediate directory not found, created one at: %s"% inter_parent_dir
    
#Create the path to the .csv file to dump all of the pct averages
csvDump_path = os.path.join(inter_parent_dir, csvDump_filename)
    
#Open the jobList document to write to
print "\nOpening and writing to joblist file: %s" % jobList_path
jobOut = open(jobList_path, 'w')

#Iterate through each subject
for subj in subjFile_paths:
    #Check to see if the output directory is present, if not create it
    subj_dir_path = os.path.join(inter_parent_dir, subj)
    if not os.path.isdir(subj_dir_path):
        os.mkdir(subj_dir_path)
    
    #Get the input files
    idxPath = subjFile_paths[subj]['idx']
    pctPath = subjFile_paths[subj]['pct']
    
    #Iterate through each component
    for comp_num in range(1,SEG_NUM+1):
        #Generate the output file paths
        binarizedComp_name = idxPath.split('/')[-1].split('.')[0]+"_comp"+str(comp_num)+"_binarized.nii.gz"
        binarizedComp_outPath = os.path.join(subj_dir_path, binarizedComp_name )
        pct_comp_name = pctPath.split('/')[-1].split('.')[0]+"_comp"+str(comp_num)+".nii.gz"
        pct_comp_outPath = os.path.join(subj_dir_path, pct_comp_name )

        #Generate the fsl commands
        binarization='fslmaths %s -thr %d -thr %d -bin %s' % (idxPath, comp_num,comp_num, binarizedComp_outPath)
        multiplication='fslmaths %s -mul %s %s' % (pctPath, binarizedComp_outPath, pct_comp_outPath)
        calAvg='echo "%s,%s,`fslstats %s -M`" >> %s' % (subj, "comp_"+str(comp_num), pct_comp_outPath, csvDump_path)
        
        #Write the commands to the jobList file
        jobOut.write(binarization+"\n")
        jobOut.write(multiplication+"\n")
        jobOut.write(calAvg+"\n")


#Close the jobList output stream
jobOut.close()
print "\nFinished writing to joblist file"

#Optionally submit the file to cluster
print "\nSubmit status: %s" % str(SUBMIT)
if SUBMIT:
    print "Submitting job..."
    cmd = ['qbatch','-w',WALLTIME, '-c',CHUNKSIZE, jobList_path]
    subprocess.call(cmd)

print "Done"    


Intermediate directory found: /data/chamal/projects/anthony/nmf_parcellation/cortical_tractMap/seg5_tract2voxel_probability_labels/tractAnalysis_model_space

Opening and writing to joblist file: /data/chamal/projects/anthony/qbatch_jobDocs/track_analysis_related/autoSubmitted_seg5_tractAnalysis.sub

Finished writing to joblist file

Submit status: False
Done
